<a href="https://colab.research.google.com/github/Serge3leo/temp-cola/blob/main/stackoverflow-79661744.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to compile the code with current versions of f2py?

[https://stackoverflow.com/questions/79661744/how-to-compile-the-code-with-current-versions-of-f2py](
https://stackoverflow.com/questions/79661744/how-to-compile-the-code-with-current-versions-of-f2py)

From F2PY reference manual:

1. `f2py` doesn’t currently support binding interface blocks. Workarounds:

    1. See [Using FYPP for binding generic interfaces](
       https://numpy.org/doc/stable/f2py/advanced/boilerplating.html);

    3. Comment all module, contains, ... statements;
  
    4. Add additional Fortran interface block (experimental, I did not
       find this in F2PY reference manual);
```
    interface iface
        module procedure spatialaverage
    end interface iface
```

2. Your code is F90 with a fixed form. You can use a `--f90flags=-ffixed-form`, but not all versions of Numpy (f2py) will work stably. It might be worth converting the file to a free form (for example, my local NumPy & Codespaces NumPy with fixed form - OK, but Colab NumPy - not).

Since using FYPP in conjunction with Fortran magic is difficult, only workarounds 1.B and 1.C are shown in this Jupiter Notebook.

In [1]:
try:
    %load_ext fortranmagic
except ModuleNotFoundError:
    # Example of install required packages for know enviroments
    if 'codespace' in str(get_ipython().config):
        !conda install -y -q -c conda-forge fortran-compiler
        %pip install -q -U fortran-magic 'setuptools>=76.1' meson \
                charset-normalizer ninja cmake pkgconfig
        print("WARNING: NEED RESTART")
        %load_ext fortranmagic
    elif 'google.colab' in str(get_ipython().config):
        %pip install -q -U fortran-magic meson \
                charset-normalizer ninja cmake pkgconfig
        %load_ext fortranmagic
        %fortran_config --backend meson -v
    else:
        assert False, """Can't load fortranmagic. For install:
                      pip install -U fortran-magic
                      or
                      conda install -c conda-forge fortran-magic
                      See: https://github.com/mgaitan/fortran_magic/blob/master/documentation.ipynb
                      """

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.4 MB/s eta 0:00:00


New default arguments for %fortran:
	--backend meson -v


In [2]:
import sys
import pprint

def get_mod_inf(fo_name: str, fn_name: str = None) -> None:
    for mn in sys.modules.keys():
        if "_fortran_magic_" in mn:
            md = sys.modules[mn].__dict__
            if fo_name in md:
                pprint.pprint(md)
                fo = md[fo_name]
                print(f"\nFortran object of {fo_name}")
                pprint.pprint(fo.__dict__)
                if fn_name in fo.__dict__:
                    print(f"\nFortran function {fn_name}")
                    pprint.pprint(fo.__dict__[fn_name].__dict__)

In [3]:
import numpy as np
import numpy.f2py as f2py
print(f"Python: {sys.version=}")
print(f"Numpy: {np.__version__=} {f2py.__version__.version=}")

Python: sys.version='3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]'
Numpy: np.__version__='2.0.2' f2py.__version__.version='2.0.2'


## Comment all module, contains, ... statements

In [4]:
%%fortran -v
! f2py doesn’t currently support binding interface blocks
! See: https://numpy.org/doc/stable/f2py/advanced/boilerplating.html
!
!      module mod_spatialaverage
!      contains

subroutine spatialaverage(depdata,vardata,depval,varout,sum_a,sum_b,nz,nx,ny)

      implicit none
      real, intent(in) :: depdata(nz,ny,nx)
      real, intent(in) :: vardata(nz,ny,nx)
      integer, intent(in) :: nz,nx,ny
      real, intent(in) :: depval
      real, intent(out) :: varout(ny,nx)
      real, intent(out) :: sum_a(ny,nx),sum_b(ny,nx)
      integer ii,jj,ilev
      real missing_value,dz
      ! ----------------------------------
      varout=0.0;
      missing_value=-1.e3
      ! ----------------------------------
      do ii = 1,nx
        do jj= 1,ny
        ! if it is landpoint:
        if (depdata(1,jj,ii).le.missing_value) cycle
        ! --------------------------------
          dz=abs(0.0-depdata(1,jj,ii));
          sum_a(jj,ii)=sum_a(jj,ii)+vardata(1,jj,ii)*dz
          sum_b(jj,ii)=sum_b(jj,ii)+dz;
          ! --------------------------------
          do ilev=2,nz
            dz=abs(depdata(ilev,jj,ii)-depdata(ilev-1,jj,ii));
            if (depdata(ilev,jj,ii).gt.depval) then
              sum_a(jj,ii)=sum_a(jj,ii)+vardata(ilev,jj,ii)*dz
              sum_b(jj,ii)=sum_b(jj,ii)+dz;
            endif
            if (depdata(ilev,jj,ii).lt.depval.and.depdata(ilev-1,jj,ii).gt.depval) then
              dz=abs(depval-depdata(ilev-1,jj,ii))
              sum_a(jj,ii)=sum_a(jj,ii)+vardata(ilev,jj,ii)*dz
              sum_b(jj,ii)=sum_b(jj,ii)+dz;
            endif
          enddo
        ! --------------------------------
        enddo
      enddo
      varout=sum_a/sum_b;
      ! ==================================
      return
end subroutine spatialaverage

!      end module mod_spatialaverage
      ! ==================================================


Ok. The following fortran objects are ready to use: spatialaverage


In [5]:
print(spatialaverage.__doc__)

varout,sum_a,sum_b = spatialaverage(depdata,vardata,depval,[nz,nx,ny])

Wrapper for ``spatialaverage``.

Parameters
----------
depdata : input rank-3 array('f') with bounds (nz,ny,nx)
vardata : input rank-3 array('f') with bounds (nz,ny,nx)
depval : input float

Other Parameters
----------------
nz : input int, optional
    Default: shape(depdata, 0)
nx : input int, optional
    Default: shape(depdata, 2)
ny : input int, optional
    Default: shape(depdata, 1)

Returns
-------
varout : rank-2 array('f') with bounds (ny,nx)
sum_a : rank-2 array('f') with bounds (ny,nx)
sum_b : rank-2 array('f') with bounds (ny,nx)



In [6]:
get_mod_inf('spatialaverage')

{'__doc__': "This module '_fortran_magic_856cc0f567ef1f8b060a3c77fe2b3fb4' is "
            'auto-generated with f2py (version:2.0.2).\n'
            'Functions:\n'
            '    varout,sum_a,sum_b = '
            'spatialaverage(depdata,vardata,depval,nz=shape(depdata, '
            '0),nx=shape(depdata, 2),ny=shape(depdata, 1))\n'
            '.',
 '__f2py_numpy_version__': '2.0.2',
 '__file__': '/root/.cache/ipython/fortranmagic/18031103/_fortran_magic_856cc0f567ef1f8b060a3c77fe2b3fb4.cpython-311-x86_64-linux-gnu.so',
 '__fortran_magic_856cc0f567ef1f8b060a3c77fe2b3fb4_error': <class '_fortran_magic_856cc0f567ef1f8b060a3c77fe2b3fb4.error'>,
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader object at 0x7cf4bc661690>,
 '__name__': '_fortran_magic_856cc0f567ef1f8b060a3c77fe2b3fb4',
 '__package__': '',
 '__spec__': ModuleSpec(name='_fortran_magic_856cc0f567ef1f8b060a3c77fe2b3fb4', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x7cf4bc661690>, origin='/r

## Add additional Fortran interface block (experimental)

In [7]:
%%fortran -v
! f2py doesn’t currently support binding interface blocks
! See: https://numpy.org/doc/stable/f2py/advanced/boilerplating.html
!
module mod_spatialaverage
    interface iface
        module procedure spatialaverage
    end interface iface

    contains

    subroutine spatialaverage(depdata,vardata,depval,varout,sum_a,sum_b,nz,nx,ny)
      implicit none
      real, intent(in) :: depdata(nz,ny,nx)
      real, intent(in) :: vardata(nz,ny,nx)
      integer, intent(in) :: nz,nx,ny
      real, intent(in) :: depval
      real, intent(out) :: varout(ny,nx)
      real, intent(out) :: sum_a(ny,nx),sum_b(ny,nx)
      integer ii,jj,ilev
      real missing_value,dz
      ! ----------------------------------
      varout=0.0;
      missing_value=-1.e3
      ! ----------------------------------
      do ii = 1,nx
        do jj= 1,ny
        ! if it is landpoint:
        if (depdata(1,jj,ii).le.missing_value) cycle
        ! --------------------------------
          dz=abs(0.0-depdata(1,jj,ii));
          sum_a(jj,ii)=sum_a(jj,ii)+vardata(1,jj,ii)*dz
          sum_b(jj,ii)=sum_b(jj,ii)+dz;
          ! --------------------------------
          do ilev=2,nz
            dz=abs(depdata(ilev,jj,ii)-depdata(ilev-1,jj,ii));
            if (depdata(ilev,jj,ii).gt.depval) then
              sum_a(jj,ii)=sum_a(jj,ii)+vardata(ilev,jj,ii)*dz
              sum_b(jj,ii)=sum_b(jj,ii)+dz;
            endif
            if (depdata(ilev,jj,ii).lt.depval.and.depdata(ilev-1,jj,ii).gt.depval) then
              dz=abs(depval-depdata(ilev-1,jj,ii))
              sum_a(jj,ii)=sum_a(jj,ii)+vardata(ilev,jj,ii)*dz
              sum_b(jj,ii)=sum_b(jj,ii)+dz;
            endif
          enddo
        ! --------------------------------
        enddo
      enddo
      varout=sum_a/sum_b;
      ! ==================================
      return
    end subroutine spatialaverage
end module mod_spatialaverage


Ok. The following fortran objects are ready to use: mod_spatialaverage


In [8]:
print(mod_spatialaverage.spatialaverage.__doc__)

varout,sum_a,sum_b = spatialaverage(depdata,vardata,depval,[nz,nx,ny])

Wrapper for ``spatialaverage``.

Parameters
----------
depdata : input rank-3 array('f') with bounds (nz,ny,nx)
vardata : input rank-3 array('f') with bounds (nz,ny,nx)
depval : input float

Other Parameters
----------------
nz : input int, optional
    Default: shape(depdata, 0)
nx : input int, optional
    Default: shape(depdata, 2)
ny : input int, optional
    Default: shape(depdata, 1)

Returns
-------
varout : rank-2 array('f') with bounds (ny,nx)
sum_a : rank-2 array('f') with bounds (ny,nx)
sum_b : rank-2 array('f') with bounds (ny,nx)



In [9]:
get_mod_inf('mod_spatialaverage')

{'__doc__': "This module '_fortran_magic_576bb8e5cc608fd5705367ca31a62194' is "
            'auto-generated with f2py (version:2.0.2).\n'
            'Functions:\n'
            'Fortran 90/95 modules:\n'
            '  mod_spatialaverage --- spatialaverage().',
 '__f2py_numpy_version__': '2.0.2',
 '__file__': '/root/.cache/ipython/fortranmagic/18031103/_fortran_magic_576bb8e5cc608fd5705367ca31a62194.cpython-311-x86_64-linux-gnu.so',
 '__fortran_magic_576bb8e5cc608fd5705367ca31a62194_error': <class '_fortran_magic_576bb8e5cc608fd5705367ca31a62194.error'>,
 '__loader__': <_frozen_importlib_external.ExtensionFileLoader object at 0x7cf4bc5c56d0>,
 '__name__': '_fortran_magic_576bb8e5cc608fd5705367ca31a62194',
 '__package__': '',
 '__spec__': ModuleSpec(name='_fortran_magic_576bb8e5cc608fd5705367ca31a62194', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0x7cf4bc5c56d0>, origin='/root/.cache/ipython/fortranmagic/18031103/_fortran_magic_576bb8e5cc608fd5705367ca31a62194.cpyt